In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import itertools
import requests
import datetime
from bs4 import BeautifulSoup
import pandas as pd
from pandas import ExcelWriter
import time
import unicodedata
import threading
import pyodbc
import os
import itertools

In [2]:
fecha = datetime.date.today()

In [3]:
url_base = "https://www.dafiti.cl/femenino/vestuario/"

In [4]:
def createDriver():
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument(f'user-agent={user_agent}')
    chrome_options.add_argument("--window-size=1625x2444")
    # options.headless = True
    return webdriver.Chrome(options = chrome_options)

In [5]:
def aplanar_lista(lista):
    if type(lista[0][0]) != list:
        return lista
    else:
        lista = list(itertools.chain(*lista))
        return aplanar_lista(lista)


In [6]:
browser = createDriver()
browser.get(url_base)

In [7]:
element_a = browser.find_element_by_class_name('fct-color-list').find_elements_by_tag_name('a')
auxiliar_colores = [[element.text,element.get_attribute('href')] for element in element_a]

In [8]:
hrefs_list = []
skip = []
for auxiliar in auxiliar_colores:
    try:
        browser.get(auxiliar[1])
    except:
        browser.quit()
        time.sleep(1)
        browser = createDriver()
        skip.append(auxiliar[1])
        continue
        
    while True:
        for i in browser.find_elements_by_class_name('itm-link'):
            
            hrefs_list.append([int(i.get_attribute('id').split(':')[0]),
                               i.get_attribute('href'),
                               auxiliar[0],
                               auxiliar[1]                              
                              ])
        try:
            WebDriverWait(browser, 50).until(EC.presence_of_element_located((By.ID,'pagingNext'))).click()
        except:
            break    

In [9]:
len(hrefs_list)

18389

In [10]:
df = pd.DataFrame(hrefs_list)
df = df.rename(columns={0:'pos',
                        1:'url',
                        2:'color',
                        3:'pagina_scraper'})

In [11]:
batch_size = 5
url_chunks = [df[x:x+batch_size] for x in range(0, len(df), batch_size)]


In [23]:
def scrape_url(url_aux,pos_aux,url_scraper,aux_color):
    
    lista_auxiliar = []
    try:
        soup = BeautifulSoup(requests.get(url_aux).text,'html.parser')
    except:
        return [-1]
    for i in soup.find_all(class_='item-box-holder'):
    
        lista_auxiliar.append([
            pos_aux,
            i['data-specialprice'],
            i['data-price'],
            i['data-originalprice'],
            i.find(class_='selectSize mbs')['data-sku'],
            aux_color,
            soup.find(class_='prd-brand mbs').text,#MARCA
            soup.find(class_='prd-title').text,#DESC
            soup.find(class_='prd-title')['data-categories'],#TIPO
            soup.find(id='productZoom')['data-zoom-image'],#JPG
            url_aux,
            url_scraper
            ])
    
    return lista_auxiliar

def scrape_batch(url_chunk):
    chunk_resp = []
    for indices,fila in url_chunk.iterrows():
        chunk_resp.append(scrape_url(fila['url'],fila['pos'],fila['pagina_scraper'],fila['color']))
                     
    return chunk_resp

In [13]:
start_ = datetime.datetime.now()
items = []
var = 0
for url_chunk in url_chunks:
    items.append(scrape_batch(url_chunk))
    var += 1
    
    if var == 50:
        var = 0
        try:
            to_xlsx = pd.DataFrame(items)
            to_xlsx.to_excel('dafiti_cl.xlsx')
                                            
            file = open("dafiti_cl.txt", "w")
            file.write( str(url_chunk) + os.linesep)
            file.close()
        except:
            print("Error al crear .xlsx")
        
    
end_ = datetime.datetime.now()

ConnectionError: HTTPSConnectionPool(host='www.dafiti.cl', port=443): Max retries exceeded with url: /Cuadro-Lady-Genny-Negro---Calce-Ajustado-799180.html (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f718e7a05c0>: Failed to establish a new connection: [Errno 101] Network is unreachable',))

'https://www.dafiti.cl/Cuadro-Lady-Genny-Negro---Calce-Ajustado-799180.html'

In [48]:
url_aux = url_chunks[1520:].copy()

In [49]:
#start_ = datetime.datetime.now()
#items = []
var = 0
for url_chunk in url_aux:
    items.append(scrape_batch(url_chunk))
    var += 1
    
    if var == 50:
        var = 0
        try:
            to_xlsx = pd.DataFrame(items)
            to_xlsx.to_excel('dafiti_cl.xlsx')
                                            
            file = open("dafiti_cl.txt", "w")
            file.write( str(url_chunk) + os.linesep)
            file.close()
        except:
            print("Error al crear .xlsx")
        
    
#end_ = datetime.datetime.now()

In [20]:
url_chunk.iloc[-1]

pos                                                              21
url               https://www.dafiti.cl/Polera-Only-Alicia-Blanc...
color                                                      Amarillo
pagina_scraper           https://www.dafiti.cl/femenino/vestuario/#
Name: 7604, dtype: object

In [22]:
items[-1]

[[[12,
   '39990.00',
   '39990.00',
   '44990.00',
   'RO747AT99TDGCL',
   'Amarillo',
   'Rockford',
   'Blusa Manga Larga Mujer Amanda Crudo Rockford',
   'Ropa Mujer|Blusas|Blusas Manga Larga',
   'https://dafitistaticcl-a.akamaihd.net/p/rockford-7924-000938-1-zoom.jpg',
   'https://www.dafiti.cl/Blusa-Manga-Larga-Mujer-Amanda-Crudo-Rockford-839000.html',
   'https://www.dafiti.cl/femenino/vestuario/#']],
 [[13,
   '12990.00',
   '12990.00',
   '15990.00',
   'LO398AT04WZZCL',
   'Amarillo',
   'Lorenzo Di Pontti',
   'Blusa Manga Corta Crudo Lisa Lorenzo Di Pontti',
   'Ropa Mujer|Blusas|Blusas Manga Corta',
   'https://dafitistaticcl-a.akamaihd.net/p/lorenzo-di-pontti-9736-591958-1-zoom.jpg',
   'https://www.dafiti.cl/Blusa-Manga-Corta-Crudo-Lisa-Lorenzo-Di-Pontti-859195.html',
   'https://www.dafiti.cl/femenino/vestuario/#']],
 [[14,
   '',
   '24990.00',
   '24990.00',
   'LE210AT36EFJCL',
   'Amarillo',
   'Leonisa',
   'Brasier Control Multicolor Leonisa',
   'Ropa Mujer|Ropa

In [ ]:
new_list = aplanar_lista(items)

In [ ]:
df = pd.DataFrame(new_list)

In [ ]:
len(df.dropna()) == len(df.drop_duplicates())

In [ ]:
df.rename(columns={0:'pos',1:'precio_especial',2:'precio_dto',3:'precio',4:'id_producto',5:'color',6:'marca',
                   7:'descripcion',8:'tipo',9:'img',10:'url',11:'pagina_scraper'},
         inplace=True)

In [ ]:
df = df.drop('precio_especial',axis=1)

In [ ]:
df.to_excel(f'dafiti_cl-{fecha}.xlsx')


In [14]:
len(items)

1520

In [ ]:
### https://mx.andrea.com/mujer/ropa

'29'